# 09 - BAM for Bayesian Surprise

**Purpose**: This notebook implements a Bayesian Assurance Model (BAM) simulation for planning new studies to determine the sample size needed to achieve a target level of Bayesian surprise, as measured by the Kullback–Leibler (KL) divergence between a prior and posterior distribution.

**Inputs**: None (this notebook uses simulated data based on theoretical parameters).

**Outputs**:
- A plot showing the relationship between sample size and both the mean Bayesian surprise and the probability of achieving the target surprise level.
- Console output indicating the minimum sample size required to achieve 80% power (assurance) for the specified surprise threshold.

### 9.1 BAM Simulation for Bayesian Surprise

This cell defines the functions and runs the complete simulation...


In [ ]:
# %reload_ext autoreload
# %autoreload 2


import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
from scipy.special import kl_div

def calculate_bayesian_surprise(prior_samples, posterior_samples):
    """Calculate Bayesian surprise using KL divergence"""
    # Estimate densities for prior and posterior
    prior_density, bins = np.histogram(prior_samples, bins=30, density=True)
    posterior_density, _ = np.histogram(posterior_samples, bins=bins, density=True)

    # Avoid division by zero in KL divergence
    prior_density = np.clip(prior_density, 1e-10, None)
    posterior_density = np.clip(posterior_density, 1e-10, None)

    # Calculate KL divergence
    surprise = np.sum(kl_div(posterior_density, prior_density))
    return surprise

def simulate_sample_size_for_surprise(min_sample_size=10, max_sample_size=100,
                                     step_size=10, min_surprise=0.5,
                                     true_param=0.65, prior_alpha=1, prior_beta=1,
                                     n_sims=100):
    """
    Determine sample size needed to achieve a target Bayesian surprise level

    Parameters:
    -----------
    min_sample_size, max_sample_size, step_size: range of sample sizes to test
    min_surprise: minimum Bayesian surprise threshold
    true_param: true parameter value for simulation
    prior_alpha, prior_beta: parameters of Beta prior
    n_sims: number of simulations per sample size

    Returns:
    --------
    Dictionary with sample sizes and corresponding surprise values
    """
    sample_sizes = range(min_sample_size, max_sample_size + 1, step_size)
    results = {'sample_size': [], 'mean_surprise': [], 'std_surprise': [],
               'prop_above_threshold': []}

    # Generate prior samples once (same for all sample sizes)
    prior_samples = np.random.beta(prior_alpha, prior_beta, size=10000)

    for n in sample_sizes:
        surprise_values = []

        for _ in range(n_sims):
            # Generate simulated data based on true parameter
            data = np.random.binomial(1, true_param, size=n)

            # Build and sample from the posterior
            with pm.Model() as model:
                theta = pm.Beta('theta', alpha=prior_alpha, beta=prior_beta)
                obs = pm.Bernoulli('obs', p=theta, observed=data)
                posterior = pm.sample(1000, tune=1000, chains=2, progressbar=False)

            # Extract posterior samples
            posterior_samples = posterior.posterior['theta'].values.flatten()

            # Calculate Bayesian surprise
            surprise = calculate_bayesian_surprise(prior_samples, posterior_samples)
            surprise_values.append(surprise)

        # Store results
        results['sample_size'].append(n)
        results['mean_surprise'].append(np.mean(surprise_values))
        results['std_surprise'].append(np.std(surprise_values))
        results['prop_above_threshold'].append(
            np.mean([s >= min_surprise for s in surprise_values]))

    return results

# Run the simulation
np.random.seed(42)
results = simulate_sample_size_for_surprise(min_sample_size=10, max_sample_size=100,
                                           step_size=10, min_surprise=0.5)

# Plot the results
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.errorbar(results['sample_size'], results['mean_surprise'],
             yerr=results['std_surprise'], marker='o')
plt.axhline(y=0.5, color='r', linestyle='--', label='Target surprise threshold')
plt.xlabel('Sample Size')
plt.ylabel('Mean Bayesian Surprise (KL divergence)')
plt.title('Mean Bayesian Surprise vs Sample Size')
plt.grid(True)
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(results['sample_size'], results['prop_above_threshold'], marker='o')
plt.axhline(y=0.8, color='r', linestyle='--', label='80% power threshold')
plt.xlabel('Sample Size')
plt.ylabel('Proportion of Simulations Above Threshold')
plt.title('Probability of Achieving Target Surprise vs Sample Size')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Find the minimum sample size that achieves 80% power
min_required_n = next((n for n, prop in zip(results['sample_size'],
                                          results['prop_above_threshold'])
                      if prop >= 0.8), None)

print(f"Minimum sample size required: {min_required_n}")


In [ ]:
import numpy as np
from scipy.special import psi, gammaln
import matplotlib.pyplot as plt


def beta_kl_divergence(alpha1, beta1, alpha2, beta2):
    """Calculate KL divergence between two Beta distributions using vectorized operations"""
    return (
        gammaln(alpha1 + beta1) - gammaln(alpha2 + beta2)
        + gammaln(alpha2) + gammaln(beta2) - gammaln(alpha1) - gammaln(beta1)
        + (alpha1 - alpha2) * psi(alpha1)
        + (beta1 - beta2) * psi(beta1)
        + (alpha2 - alpha1 + beta2 - beta1) * psi(alpha1 + beta1)
    )


def diagnostic_sample_size_simulation(prior_alpha, prior_beta,
                                      disease_prev, normal_prev,
                                      threshold=0.2,
                                      min_n=10, max_n=200, step=10,
                                      n_sims=500, power=0.8):
    """
    Bayesian sample size estimation for diagnostic surprise model

    Parameters:
        prior_alpha, prior_beta: Beta prior parameters for normal class
        disease_prev: True prevalence in disease population (0-1)
        normal_prev: True prevalence in normal population (0-1)
        threshold: Surprise cutoff for classification
        min_n, max_n, step: Sample size range to test
        n_sims: Number of simulations per sample size
        power: Required sensitivity/specificity probability

    Returns:
        Sample size results and optimal N
    """

    sample_sizes = range(min_n, max_n+1, step)
    results = {
        'n': [],
        'sensitivity': [],
        'specificity': []
    }

    for n in sample_sizes:
        disease_surprise = []
        normal_surprise = []

        # Simulate disease cases
        for _ in range(n_sims):
            data = np.random.binomial(1, disease_prev, n)
            successes = data.sum()
            post_alpha = prior_alpha + successes
            post_beta = prior_beta + (n - successes)
            surprise = beta_kl_divergence(post_alpha, post_beta,
                                        prior_alpha, prior_beta)
            disease_surprise.append(surprise)

        # Simulate normal cases
        for _ in range(n_sims):
            data = np.random.binomial(1, normal_prev, n)
            successes = data.sum()
            post_alpha = prior_alpha + successes
            post_beta = prior_beta + (n - successes)
            surprise = beta_kl_divergence(post_alpha, post_beta,
                                        prior_alpha, prior_beta)
            normal_surprise.append(surprise)

        # Calculate classification metrics
        sens = np.mean(np.array(disease_surprise) <= threshold)
        spec = np.mean(np.array(normal_surprise) > threshold)

        results['n'].append(n)
        results['sensitivity'].append(sens)
        results['specificity'].append(spec)

    # Find optimal sample size
    # In diagnostic_sample_size_simulation()
    optimal_n = None
    for i, (n, sens, spec) in enumerate(zip(results['n'],
                                          results['sensitivity'],
                                          results['specificity'])):
        if sens >= power and spec >= power:
            optimal_n = n
            break  # Return first n meeting criteria

    # If no n meets criteria,
    # Find closest sample size meeting criteria
    if optimal_n is None:
        # Use indices instead of sample size values
        indices = range(len(results['n']))

        closest_idx = min(indices,
                         key=lambda i: (
                             abs(results['sensitivity'][i] - power) +
                             abs(results['specificity'][i] - power)
                         ))

        closest_n = results['n'][closest_idx]
        print(f"No sample size achieved {power*100}% power. Closest: {closest_n}")
        print(f"Sensitivity: {results['sensitivity'][closest_idx]:.2f}")
        print(f"Specificity: {results['specificity'][closest_idx]:.2f}")


    # Plot results
    # Plot results
    plt.figure(figsize=(10,6))
    plt.plot(results['n'], results['sensitivity'], label='Sensitivity')
    plt.plot(results['n'], results['specificity'], label='Specificity')
    plt.axhline(0.8, color='gray', linestyle='--')

    # Add vertical line only if optimal_n exists
    if optimal_n is not None:
        plt.axvline(optimal_n, color='red', linestyle=':',
                    label=f'Optimal N: {optimal_n}')
        print(f"Optimal sample size: {optimal_n}")
    else:
        print("Warning: No sample size achieved required power")
        plt.axvline((min_n + max_n)//2, color='yellow', linestyle='--',
                    label='No optimal N found')

    plt.xlabel('Sample Size')
    plt.ylabel('Probability')
    plt.title('Diagnostic Classification Performance vs Sample Size')
    plt.legend()
    plt.grid(True)
    plt.show()
    return results, optimal_n

# Example Usage
prior_alpha = 2  # Prior based on normal population (mean=0.2)
prior_beta = 8

results, optimal_n = diagnostic_sample_size_simulation(
    prior_alpha=prior_alpha,
    prior_beta=prior_beta,
    disease_prev=0.1,   # Disease population prevalence
    normal_prev=0.9,   # Normal population prevalence
    threshold=1,
    min_n=50,
    max_n=1000,
    step=10
)




print(f"Optimal sample size: {optimal_n}")
